In [3]:
import glob, os, numpy as np
import theano as th, tensorflow as tf
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

In [4]:
path = '/home/kristina/Diploma/Data/Data/dataset/images/lab/'
category = []
for i in glob.glob(path + '*'):
    category.append(i[len(path):])

In [5]:
def load_images(path):
    images, answers = [], []
    size = 32, 32
    group = 0
    for item in category:
        for infile in glob.glob(path + item + '/*.jpg'):
            im = Image.open(infile)
            im = im.resize(size)
            images.append(np.array(im))
            answers.append(group)
        group += 1   
    return (images, answers)

In [6]:
data = load_images(path)
X_train, X_test, y_train, y_test = train_test_split(data[0], data[1], test_size = 0.25, random_state = 42)
X_train = np.array(X_train)/255
X_test = np.array(X_test)/255

In [24]:
nb_classes = len(category)           # Количество классов изображений                     
nb_epoch = 30                        # Количество эпох для обучения
img_rows, img_cols = 32, 32          # Размер изображений
batch_size = 32                      # Размер мини-выборки

In [16]:
# for i in range(nb_classes):
#     print(str(i) + ' - ' + str(y_test.count(i)) + ' - ' + str(y_train.count(i)))

In [17]:
# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [21]:
# Создаем последовательную модель
model = Sequential()
# Первый сверточный слой
model.add(Convolution2D(32, 3, 3, border_mode = 'same', input_shape = (32, 32, 3), activation = 'relu'))
# Второй сверточный слой
model.add(Convolution2D(32, 3, 3, activation = 'relu'))
# Первый слой подвыборки
model.add(MaxPooling2D(pool_size = (2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.25))

In [22]:
# Третий сверточный слой
model.add(Convolution2D(64, 3, 3, border_mode = 'same', activation = 'relu'))
# Четвертый сверточный слой
model.add(Convolution2D(64, 3, 3, activation = 'relu'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size = (2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.25))
# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(512, activation = 'relu'))
# Слой регуляризации Dropout
model.add(Dropout(0.5))
# Выходной полносвязный слой
model.add(Dense(nb_classes, activation = 'softmax'))

In [25]:
# Задаем параметры оптимизации
sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
# Обучаем модель
X_train = X_train.reshape(len(X_train), 32, 32, 3)
model.fit(X_train, Y_train, batch_size = batch_size, nb_epoch = nb_epoch, validation_split = 0.1, shuffle = True)
# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(np.array(X_test).reshape(len(X_test), 32, 32, 3), Y_test, verbose = 0)
print('Точность работы на тестовых данных: %.2f%%' % (scores[1]*100))

Train on 15624 samples, validate on 1736 samples
Epoch 1/30
15624/15624 [==============================] - 97s - loss: 0.4629 - acc: 0.8434 - val_loss: 2.5159 - val_acc: 0.4925